In [1]:
import sys
import os
import pandas as pd
import re

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [3]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")


In [4]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [5]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [6]:
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']

In [7]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [8]:
test_train_df

,sub_label,SplitGroup,AssignedInJune,SubjectId,Task,Include
1,DEV004,Holdout,True,NaN,NaN,True
2,DEV005,Train,True,NaN,NaN,True
3,DEV006,Train,True,NaN,NaN,True
4,DEV008,Holdout,True,NaN,NaN,True
5,DEV009,Train,True,NaN,NaN,True
...,...,...,...,...,...,...
184,DEV221,Train,False,NaN,NaN,True
185,DEV222,Holdout,False,NaN,NaN,True
186,DEV223,Holdout,False,NaN,NaN,True
187,DEV224,Holdout,False,NaN,NaN,True


In [9]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [10]:
from dev_wtp_io_utils import import_sst_cond_w1_subjs_to_pkl

In [14]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions',
    out_folder = ml_data_folderpath + "/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
1
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/Brain_Data_conditions_2subs.pkl


OK, not bad, so we have learned to load data, and show it. We might be best to re-write the code, though, to use the first image as the image series's own mask--using `nilearn.masking.compute_brain_mask`, rather than worry about why the input mask isn't quite aligned. 

In [15]:
len(train_subjs)

93

In [12]:
import_sst_cond_w1_subjs_to_pkl(
     train_subjs[0:6],'conditions',
    out_folder = ml_data_folderpath + "/SST/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
......verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continu

In [14]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:40],'conditions',
    out_folder = ml_data_folderpath + "/SST/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
......verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continu

### Selected conditions only

In [47]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
1
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_2subs_correct_cond.pkl


In [48]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:43],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mas

#### With main set of ~90 Subjects

In [11]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [12]:
", ".join(train_subjs_selected)

'DEV005, DEV006, DEV009, DEV010, DEV012, DEV013, DEV014, DEV015, DEV016, DEV017, DEV018, DEV019, DEV021, DEV022, DEV023, DEV024, DEV025, DEV026, DEV027, DEV028, DEV029, DEV030, DEV034, DEV035, DEV036, DEV039, DEV040, DEV041, DEV042, DEV043, DEV046, DEV048, DEV049, DEV050, DEV051, DEV052, DEV053, DEV055, DEV056, DEV057, DEV058, DEV059, DEV060, DEV062, DEV068, DEV069, DEV071, DEV073, DEV074, DEV076, DEV077, DEV079, DEV083, DEV084, DEV085, DEV086, DEV087, DEV089, DEV011, DEV020, DEV047, DEV064, DEV067, DEV097, DEV105, DEV107, DEV135, DEV141, DEV145, DEV157, DEV158, DEV159, DEV164, DEV169, DEV171, DEV173, DEV177, DEV186, DEV197, DEV206, DEV215, DEV216, DEV217, DEV218'

In [13]:
ml_data_folderpath

'/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml'

In [18]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mas

In [28]:
test_file = pickle.load(open('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_84subs_correct_cond.pkl','rb'))


In [31]:
len(np.unique(test_file.X.subject))

81

### With supplementary data

In [14]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1
#individual_differences['wave'] = individual_differences['wave'].astype(object) # for compatibility with the wave column in the dataset

In [15]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:43],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_CorrectStopWithSupplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask c

In [18]:
test_file = pickle.load(open('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_43subs_CorrectStopWithSupplementary.pkl','rb'))

In [15]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:4],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mas

In [16]:
test_file = pickle.load(open('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_4subs_correct_cond.pkl','rb'))

In [20]:
", ".join(test_file.X.columns)

'condition_index, condition_label, raw_beta_description, beta, subject, wave, cancer_promoting_minus_preventing_FCI, cancer_promoting_minus_preventing_FFQ, cancer_promoting_minus_preventing_craved_FCI, cancer_promoting_minus_preventing_craved_FFQ, cancer_promoting_minus_preventing_liked_FCI, cancer_promoting_minus_preventing_liked_FFQ, cancer_promoting_FCI, cancer_promoting_FFQ, cancer_preventing_FCI, cancer_preventing_FFQ, BSCS, cSES, EDM, BIS_11, PCS, RS, TRSQ, bf_1, weight_0, height_0, birthsex, age365, bmi_0, bmi_1, bmi, bf_1_controlled, bf_1_bsexnormedzs, ACES_sum, ACES_household_dysfunction, ACES_neglectful_parenting, ACES_abuse, ACES_divorced_separated, BFI_agreeableness, BFI_conscientiousness, BFI_extraversion, BFI_neuroticism, BFI_openness, DEMO_mcarthur_social_standing, IMI_effort_importance, IMI_interest_enjoyment, IMI_perceived_choice, IMI_perceived_competence, IMI_value_usefulness, IPAQ_moderateminutes, IPAQ_sittinghours, IPAQ_vigorousminutes, IPAQ_walkingminutes, NCS_get_

In [21]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mas

During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV036
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV039
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV040
.....verified that all expected betas exist! Processing...setti

...imported data.
temp mask deleted
DEV068
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV069
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV071
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp ma

In [32]:
test_file = pickle.load(open('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_84subs_correct_cond_w_supplementary.pkl','rb'))

In [38]:
len(np.unique(test_file.X.subject))

81

In [37]:
len(np.unique(test_file.X.subject[np.isnan(test_file.X.TESQ_E_sum)==False]))

75

In [41]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_stop_w_supplementary'
)

import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_go_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask c

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV039
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV040
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take 

During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV069
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV071
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV073
....verified that all expected betas exist! Processing...setting 

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV171
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV173
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take 

During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV049
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV050
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV051
....verified that all expected betas exist! Processing...setting 

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV085
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV086
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take 

During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV217
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV218
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


## post-error conditions

In [15]:
def import_sst_cond_w1_subjs_to_pkl(subjs,first_level_fileid,out_folder = '../data/', 
                                    conditions_to_include=None,
                                    condition_count_required=None,
                                    supplementary_df = None,
                                         out_file_suffix =''):
    ## get a list of the subject folders
    sst_wt_repo = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/'
    first_level_path = sst_wt_repo + first_level_fileid + "/"
    print(first_level_path)
    subj_count = len(subjs)
    ## iterate through them
    subjs.sort()
    bd_dict={}
    #get the brain data from the beta files
    for sl in subjs:
        print(sl)
        
        #get the design data
        
        #load the matrix associated with this file
        print(first_level_path + 'sub-' + sl +'/SPM.mat')
        sl_mat = scipy.io.loadmat(
            first_level_path + 'sub-' + sl +'/SPM.mat',
            simplify_cells=True            
        )

        #convert from an SPM.mat file into a dataframe list of the betas
        beta_dict_list = []
        mat_betas = sl_mat['SPM']['Vbeta']
        for beta_i in range(len(mat_betas)):
            beta = mat_betas[beta_i]
            b_description = beta['descrip']
            condition_label = re.search('Sn\(1\)\s(.*)\*bf\(1\)',b_description)
            print(condition_label)
            if (condition_label is not None):
                beta_dict_list = beta_dict_list + [{
                    'condition_index':beta_i,
                    'condition_label':condition_label.group(1),
                    'raw_beta_description':beta['descrip'],
                    'beta':beta['fname']
                }]
        #    print(condition_label)
        beta_df = pd.DataFrame(beta_dict_list)

        #fill in the details related to the subject
        beta_df['subject']=sl
        beta_df['wave']=1
        
        if conditions_to_include is not None:
            #cut down conditions to just those specified
            #print(beta_df.condition_label)
            beta_df = beta_df.loc[beta_df.condition_label.isin(conditions_to_include),]
            #print(beta_df.condition_label)

        if condition_count_required is not None:
            if beta_df.shape[0]<condition_count_required:
                print("not enough conditions for subject "+ sl + ". Skipping this subject.")
                continue
        
        
        try: 
            bd = get_Brain_Data_betas_for_sub(
                sl, beta_df,
                betaseries_path = first_level_path,
                events_in_design=beta_df.shape[0],
                mask = "beta" # '/projects/sanlab/shared/spm12/canonical/MNI152_T1_1mm_brain_mask.nii'
            )
            bd_dict[sl]=bd
        except BehavioralDataNotFoundForBrainDataException:
            print("couldn't get data for subject " + sl + " because there was no matching behavioral data")

    #now append into a concatenated brain data file
    Brain_Data_allsubs = list(bd_dict.values())[0]
    for i in range(1,len(bd_dict.values())):
        print(i)
        val_to_append = list(bd_dict.values())[i]
        Brain_Data_allsubs= Brain_Data_allsubs.append(val_to_append)
        
    if supplementary_df is not None:
        Brain_Data_allsubs.X = pd.merge(Brain_Data_allsubs.X,supplementary_df,how='left',on=['subject','wave'])

    #dump
    out_filepath = (
        out_folder + 'Brain_Data_' +
        #'betaseries_' +
        first_level_fileid + '_' + str(subj_count) + 'subs' + out_file_suffix + '.pkl'
    )
    print(out_filepath)

    with open(out_filepath, 'wb') as pkl_file:
        pickle.dump(Brain_Data_allsubs,pkl_file)


In [ ]:
sl_mat = scipy.io.loadmat(
            first_level_path + 'sub-' + sl +'/SPM.mat',
            simplify_cells=True            
        )

In [16]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:3],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV005/SPM.mat
<re.Match object; span=(22, 63), match='Sn(1) CorrectGoFollowingCorrectStop*bf(1)'>
<re.Match object; span=(22, 62), match='Sn(1) CorrectGoFollowingFailedStop*bf(1)'>
<re.Match object; span=(22, 48), match='Sn(1) OtherCorrectGo*bf(1)'>
<re.Match object; span=(22, 45), match='Sn(1) CorrectStop*bf(1)'>
<re.Match object; span=(22, 44), match='Sn(1) FailedStop*bf(1)'>
<re.Match object; span=(22, 37), match='Sn(1) Cue*bf(1)'>
<re.Match object; span=(22, 42), match='Sn(1) FailedGo*bf(1)'>
None
None
None
None
None
None
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading 

In [19]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq

train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [21]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii


## Error Conditions

In [22]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq

train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectStop','FailedStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_vs_failed_stop'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mas